In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as

In [2]:
import xarray as xr

In [3]:
viirs = xr.open_dataset(r"C:\Users\Ankit\OneDrive\Desktop\Datasets_Forest_fire\updated_viirs_binary_fire_2015_2016.nc")

In [4]:
viirs.data_vars

Data variables:
    fire_label  (time, latitude, longitude) uint8 4MB ...

In [7]:
import rasterio
file_path = r"C:\Users\Ankit\OneDrive\Desktop\Datasets_Forest_fire\lulc_maps_tif\LULC_2015_clipped.tif"

In [ ]:
import rasterio
import numpy as np


# ------------------------------
# File paths (CHANGE these)
# ------------------------------
lulc_path = r"C:\path\to\LULC_2015_clipped.tif"
viirs_nc_path = r"C:\path\to\updated_viirs_binary_fire_2015_2016.nc"  # VIIRS NetCDF file

# ------------------------------
# Set your test time (YYYY-MM-DD)
# ------------------------------
testtime = "2015-05-15"

# ------------------------------
# Read LULC raster
# ------------------------------
with rasterio.open(lulc_path) as lulc_src:
    lulc = lulc_src.read(1)
    lulc_meta = lulc_src.meta
    lulc_transform = lulc_src.transform

# ------------------------------
# Read VIIRS binary from NetCDF for the given date
# ------------------------------
import xarray as xr

ds = xr.open_dataset(viirs_nc_path)
# Assuming variable is named 'binary_fire' and time coordinate is 'time'
# Adjust names if needed after checking ds
viirs_day = ds['binary_fire'].sel(time=testtime)

# Convert to numpy array
viirs_array = viirs_day.values

# ------------------------------
# Get coordinates of detections
# ------------------------------
lats = ds['lat'].values
lons = ds['lon'].values

# Mask for detections == 1
mask = viirs_array == 1
fire_lats = lats[mask]
fire_lons = lons[mask]

# ------------------------------
# Plot LULC with VIIRS fires overlaid
# ------------------------------
fig, ax = plt.subplots(figsize=(10, 8))

# Plot LULC base
lulc_img = ax.imshow(lulc, cmap='tab20', extent=(
    lulc_transform[2],
    lulc_transform[2] + lulc.shape[1] * lulc_transform[0],
    lulc_transform[5] + lulc.shape[0] * lulc_transform[4],
    lulc_transform[5]
))

# Overlay fires
ax.scatter(fire_lons, fire_lats, color='red', s=10, label='VIIRS Fires')

# Labels and legend
ax.set_title(f"LULC (2015) with VIIRS Fires on {testtime}")
ax.set_xlabel("Longitude")
ax.set_ylabel("Latitude")
ax.legend()

plt.show()
